In [324]:
import pandas as pd
import numpy as np
from torch import sigmoid
import torch
from torch.nn import BCELoss
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

from torch.distributions.multivariate_normal import MultivariateNormal

In [594]:
X,y = make_classification(n_samples= 100_000, n_features=100, n_informative=5)

In [595]:
xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=0.3,shuffle=True)

In [596]:
y.mean()

0.50005

In [597]:
lgr_ = LogisticRegression()

In [598]:
lgr_.fit(xtrain,ytrain)

LogisticRegression()

In [599]:
ypred = lgr_.predict(xtest)
p = lgr_.predict_proba(xtest)[:,1]

In [600]:
accuracy_score(ytest, ypred)

0.9523666666666667

In [601]:
roc_auc_score(y_true= ytest, y_score=p)

0.9750487431425161

In [602]:
confusion_matrix(y_true= ytest, y_pred= ypred, normalize='true')

array([[0.92793517, 0.07206483],
       [0.02342558, 0.97657442]])

In [495]:
class RandomLogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, n_samples, n_epochs, scale = 1.0):
        super().__init__()
        self.input_dim = input_dim
        self.n_samples =n_samples
        self.n_epochs = n_epochs
        self.scale = scale
        self._l1 = torch.nn.Linear(in_features= self.input_dim,
                                   out_features=self.n_samples)
        self.l1 = torch.nn.Linear(in_features= self.input_dim, out_features=1)
        self.loss = BCELoss(reduction= 'none')

    def sample_params(self,
                      w_mean ,
                      b_mean
                      ):
        w = MultivariateNormal(loc = w_mean,
                         covariance_matrix= self.scale*torch.eye(self.input_dim)
                         ).sample((self.n_samples,))
        b = torch.normal(mean = b_mean, std =1.0 * self.scale, size = (self.n_samples,))
        self._l1.weight = torch.nn.Parameter(w)
        self._l1.bias = torch.nn.Parameter(b)

    def _forward(self,x):
        _x = self._l1(x)
        return sigmoid(_x)
    
    def _fit_params_epoch(self,X,y, w_mean, b_mean):
        self.sample_params(w_mean, b_mean)
        yt = y.reshape(-1,1).repeat(1,self.n_samples)
        Xt = self._forward(X)
        losses = self.loss(Xt,yt).mean(0)
        argmin_loss = losses.argmin()
        w_mean = self._l1.weight[argmin_loss].clone().detach()
        b_mean = self._l1.bias[argmin_loss].clone().detach()
        return w_mean, b_mean

    def fit(self,X,y):
        w_mean = torch.zeros(self.input_dim)
        b_mean = torch.tensor(0.)
        for epoch in range(self.n_epochs):
            w_mean, b_mean = self._fit_params_epoch(X,y,w_mean,b_mean)
        self.l1.weight = torch.nn.Parameter(w_mean.reshape(1,-1))
        self.l1.bias = torch.nn.Parameter(b_mean.reshape(-1))
        self.is_fitted = True
        return self

    def forward(self,x):
        assert self.is_fitted
        _x = self.l1(x)
        return sigmoid(_x)
    
    def predict(self,X):
        assert self.is_fitted
        p = self(X)
        return (p>0.5).to(torch.float).reshape(-1,)

In [603]:
lgr = RandomLogisticRegression(100,1_000,20, scale=.5)

In [604]:
ytorch_train = torch.tensor(ytrain, dtype= torch.float32)
ytorch_test = torch.tensor(ytest, dtype= torch.float32)

Xtorch_train = torch.tensor(xtrain, dtype= torch.float32)
Xtorch_test = torch.tensor(xtest, dtype= torch.float32)

In [605]:
sample_idx = np.random.choice(range(len(ytrain)),size=1000, replace=False)

In [606]:
lgr.fit(Xtorch_train[sample_idx],ytorch_train[sample_idx])

RandomLogisticRegression(
  (_l1): Linear(in_features=100, out_features=1000, bias=True)
  (l1): Linear(in_features=100, out_features=1, bias=True)
  (loss): BCELoss()
)

In [607]:
ypred = lgr.predict(Xtorch_test).numpy()
p = lgr(Xtorch_test).reshape(-1,).detach().numpy()

In [608]:
accuracy_score(ytorch_test, ypred)

0.8106

In [609]:
roc_auc_score(y_true= ytorch_test, y_score=p)

0.8854154887250747

In [593]:
confusion_matrix(y_true= ytorch_test, y_pred= ypred, normalize='true')

array([[0.71765253, 0.28234747],
       [0.28164959, 0.71835041]])

In [614]:
torch.multinomial(torch.tensor([[0.3,0.7],[0.2,0.8]]), replacement=True, num_samples=1000)

tensor([[1, 0, 1,  ..., 1, 1, 1],
        [1, 0, 1,  ..., 1, 1, 1]])